In [86]:
#Importing necessary libraries
import pandas as pd
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xg
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from feature_engine import missing_data_imputers as mdi
from feature_engine import categorical_encoders as ce
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.base import TransformerMixin
% matplotlib inline
warnings.filterwarnings("ignore")

UsageError: Line magic function `%` not found.


In [2]:
df_train = pd.read_csv("H:/Learnings/Data Science/Hackathon/HR analytics/train_jqd04QH.csv")
df_test =  pd.read_csv("H:/Learnings/Data Science/Hackathon/HR analytics/test_KaymcHn.csv")

In [3]:
df_train.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [4]:
df_test.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    3388
relevent_experience          0
enrolled_university        279
education_level            395
major_discipline          2393
experience                  44
company_size              4051
company_type              4330
last_new_job               304
training_hours               0
dtype: int64

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             18359 non-null  int64  
 1   city                    18359 non-null  object 
 2   city_development_index  18359 non-null  float64
 3   gender                  14261 non-null  object 
 4   relevent_experience     18359 non-null  object 
 5   enrolled_university     18017 non-null  object 
 6   education_level         17902 non-null  object 
 7   major_discipline        15521 non-null  object 
 8   experience              18300 non-null  object 
 9   company_size            13580 non-null  object 
 10  company_type            13320 non-null  object 
 11  last_new_job            17992 non-null  object 
 12  training_hours          18359 non-null  int64  
 13  target                  18359 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15021 entries, 0 to 15020
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15021 non-null  int64  
 1   city                    15021 non-null  object 
 2   city_development_index  15021 non-null  float64
 3   gender                  11633 non-null  object 
 4   relevent_experience     15021 non-null  object 
 5   enrolled_university     14742 non-null  object 
 6   education_level         14626 non-null  object 
 7   major_discipline        12628 non-null  object 
 8   experience              14977 non-null  object 
 9   company_size            10970 non-null  object 
 10  company_type            10691 non-null  object 
 11  last_new_job            14717 non-null  object 
 12  training_hours          15021 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.5+ MB


In [7]:
df_train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [8]:
df_train['target'].groupby(df_train['gender']).count()

gender
Female     1188
Male      12884
Other       189
Name: target, dtype: int64

In [9]:
df_train['target'].groupby(df_train['relevent_experience']).count()

relevent_experience
Has relevent experience    13596
No relevent experience      4763
Name: target, dtype: int64

In [10]:
df_train['target'].groupby(df_train['enrolled_university']).count()

enrolled_university
Full time course     3187
Part time course     1171
no_enrollment       13659
Name: target, dtype: int64

In [11]:
df_train['target'].groupby(df_train['education_level']).count()

education_level
Graduate          10769
High School        2032
Masters            4319
Phd                 459
Primary School      323
Name: target, dtype: int64

In [12]:
df_train['target'].groupby(df_train['major_discipline']).count()

major_discipline
Arts                 239
Business Degree      307
Humanities           688
No Major             206
Other                343
STEM               13738
Name: target, dtype: int64

In [13]:
df_train['target'].groupby(df_train['experience']).count()

experience
1       452
10      967
11      667
12      497
13      412
14      602
15      695
16      549
17      347
18      286
19      308
2       992
20      146
3      1159
4      1250
5      1309
6      1125
7       950
8       755
9       979
<1      416
>20    3437
Name: target, dtype: int64

In [14]:
df_train['target'].groupby(df_train['company_size']).count()

company_size
10/49        1466
100-500      2698
1000-4999    1399
10000+       2044
50-99        3120
500-999       902
5000-9999     591
<10          1360
Name: target, dtype: int64

In [15]:
df_train['target'].groupby(df_train['last_new_job']).count()

last_new_job
1        7567
2        2835
3        1027
4        1038
>4       3339
never    2186
Name: target, dtype: int64

In [17]:
# changing the values > 20 to 20 and <1 to 1 as the count is less in 1 and 20 experience bucket
df_train['experience'].replace({'<1':'1','>20':'20'},inplace=True)
df_train['experience'] = df_train['experience'].astype('float')
df_test['experience'].replace({'<1':'1','>20':'20'},inplace=True)
df_test['experience'] = df_test['experience'].astype('float')

In [18]:
df_train['target'].groupby(df_train['experience']).count()

experience
1.0      868
2.0      992
3.0     1159
4.0     1250
5.0     1309
6.0     1125
7.0      950
8.0      755
9.0      979
10.0     967
11.0     667
12.0     497
13.0     412
14.0     602
15.0     695
16.0     549
17.0     347
18.0     286
19.0     308
20.0    3583
Name: target, dtype: int64

In [19]:
#Bucketing the company size featue to very small, small, medium and large
df_train['company_size'].replace({'<10':'very small','10/49':'very small','50-99':'very small',
                                  '100-500':'small','500-999':'small',
                                  '1000-4999':'medium',
                                  '5000-9999':'large','10000+':'large'},inplace=True)
df_train['company_size'] = df_train['company_size'].astype('category')

df_test['company_size'].replace({'<10':'very small','10/49':'very small','50-99':'very small',
                                  '100-500':'small','500-999':'small',
                                  '1000-4999':'medium',
                                  '5000-9999':'large','10000+':'large'},inplace=True)
df_test['company_size'] = df_test['company_size'].astype('category')

In [20]:
df_train['target'].groupby(df_train['company_size']).count()

company_size
large         2635
medium        1399
small         3600
very small    5946
Name: target, dtype: int64

In [22]:
# replacing >4 with 5 for last_new_job feature
df_train['last_new_job'].replace({'>4':'5','never': '0'},inplace=True)
df_train['last_new_job'] = df_train['last_new_job'].astype('float')
df_test['last_new_job'].replace({'>4':'5','never': '0'},inplace=True)
df_test['last_new_job'] = df_test['last_new_job'].astype('float')

In [23]:
df_train['target'].groupby(df_train['last_new_job']).count()

last_new_job
0.0    2186
1.0    7567
2.0    2835
3.0    1027
4.0    1038
5.0    3339
Name: target, dtype: int64

In [37]:
df_train['company_size'] = df_train['company_size'].astype('object')
df_test['company_size'] = df_test['company_size'].astype('object')

In [38]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             18359 non-null  int64  
 1   city                    18359 non-null  object 
 2   city_development_index  18359 non-null  float64
 3   gender                  14261 non-null  object 
 4   relevent_experience     18359 non-null  object 
 5   enrolled_university     18017 non-null  object 
 6   education_level         17902 non-null  object 
 7   major_discipline        15521 non-null  object 
 8   experience              18300 non-null  float64
 9   company_size            13580 non-null  object 
 10  company_type            13320 non-null  object 
 11  last_new_job            17992 non-null  float64
 12  training_hours          18359 non-null  int64  
 13  target                  18359 non-null  int64  
dtypes: float64(3), int64(3), object(8)
mem

In [25]:
#Encoding the categorical variables with frequency
def frequency_encode_fun(train,test):
    cols=[i+"_freq_encode" for i in ["gender","relevent_experience",
                                     "enrolled_university","education_level",
                                     "major_discipline","company_size","company_type"]]
    imp=ce.CountFrequencyCategoricalEncoder(encoding_method="frequency")
    df=imp.fit_transform(train[["gender","relevent_experience",
                                "enrolled_university","education_level",
                                "major_discipline","company_size","company_type"]])
    df.columns=cols
    train=pd.concat([train,df],axis=1)
    df=imp.transform(test[["gender","relevent_experience",
                           "enrolled_university","education_level",
                           "major_discipline","company_size","company_type"]])
    df.columns=cols
    test=pd.concat([test,df],axis=1)
    return train,test

In [26]:
#Encoding the categorical variables with mean of target
def mean_cat_encode_fun(train,test,target):
    cols=[i+"_mean_cat_encode" for i in ["gender","relevent_experience",
                                         "enrolled_university","education_level",
                                         "major_discipline","company_size","company_type"]]
    imp=ce.MeanCategoricalEncoder()
    df=imp.fit_transform(train[["gender","relevent_experience",
                                "enrolled_university","education_level",
                                "major_discipline","company_size","company_type"]],target)
    df.columns=cols
    train=pd.concat([train,df],axis=1)
    df=imp.transform(test[["gender","relevent_experience",
                           "enrolled_university","education_level",
                           "major_discipline","company_size","company_type"]])
    df.columns=cols
    test=pd.concat([test,df],axis=1)
    return train,test

In [41]:
# Validation function
def acc_fun():
    X=df_train.drop(columns=["enrollee_id","city","target"])
    y=df_train["target"]
    
    #oversample = SMOTE()
    #X_smote, y_smote = oversample.fit_resample(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
           
    pipe=Pipeline([
    ("missing_indicator",mdi.AddMissingIndicator()),
    ("mean_median",mdi.MeanMedianImputer(imputation_method="median",variables=["experience","last_new_job"])),
    ("random_imputer",mdi.CategoricalVariableImputer(variables=["gender","enrolled_university",
                                                               "education_level","major_discipline",
                                                               "company_size","company_type"]))
    ])
    
    X_train =pipe.fit_transform(X_train)
    X_test=pipe.transform(X_test)   
    
    
    X_train,X_test=frequency_encode_fun(X_train,X_test)
    X_train,X_test=mean_cat_encode_fun(X_train,X_test,y_train)
    
    
        
    cols=X_train.select_dtypes("object").columns

    train_dummy=pd.get_dummies(data=X_train,columns=cols,drop_first=True)
    test_dummy=pd.get_dummies(data=X_test,columns=cols,drop_first=True)
    
    model=lgb.LGBMClassifier()
    model.fit(train_dummy,y_train)
    lgb_pred=model.predict(test_dummy)
    
    model=cb.CatBoostClassifier()
    model.fit(train_dummy,y_train)
    cb_pred=model.predict(test_dummy)
    
    gb = GradientBoostingClassifier()
    gb.fit(train_dummy,y_train)
    gb_pred=gb.predict(test_dummy)
    
    return roc_auc_score(y_test,lgb_pred),roc_auc_score(y_test,cb_pred), roc_auc_score(y_test,gb_pred)
   

In [45]:
def prediction_fun():
    X=df_train.drop(columns=["enrollee_id","city","target"])
    y=df_train["target"]
    test_df=df_test.drop(columns=["enrollee_id","city"])
    
    pipe=Pipeline([
    ("missing_indicator",mdi.AddMissingIndicator()),
    ("mean_median",mdi.MeanMedianImputer(imputation_method="median",variables=["experience","last_new_job"])),
    ("random_imputer",mdi.CategoricalVariableImputer(variables=["gender","enrolled_university",
                                                               "education_level","major_discipline",
                                                               "company_size","company_type"]))
    ])
    
    X=pipe.fit_transform(X)
    test_df=pipe.transform(test_df)
    
    X,test_df=frequency_encode_fun(X,test_df)
    X,test_df=mean_cat_encode_fun(X,test_df,y)
    
    cols=X.select_dtypes("object").columns

    train_dummy=pd.get_dummies(data=X,columns=cols,drop_first=True)
    test_dummy=pd.get_dummies(data=test_df,columns=cols,drop_first=True)

    model=cb.CatBoostClassifier()
    model.fit(train_dummy,y)
    pred=model.predict(test_dummy)
    
    return pred

In [44]:
acc_fun()

Learning rate set to 0.03584
0:	learn: 0.6678340	total: 11.9ms	remaining: 11.9s
1:	learn: 0.6441001	total: 26.8ms	remaining: 13.4s
2:	learn: 0.6213999	total: 37.2ms	remaining: 12.4s
3:	learn: 0.6008171	total: 47.3ms	remaining: 11.8s
4:	learn: 0.5821256	total: 56.6ms	remaining: 11.3s
5:	learn: 0.5645101	total: 65ms	remaining: 10.8s
6:	learn: 0.5490022	total: 77.4ms	remaining: 11s
7:	learn: 0.5345104	total: 85ms	remaining: 10.5s
8:	learn: 0.5210274	total: 93ms	remaining: 10.2s
9:	learn: 0.5093179	total: 97.8ms	remaining: 9.68s
10:	learn: 0.4988172	total: 105ms	remaining: 9.48s
11:	learn: 0.4889522	total: 115ms	remaining: 9.43s
12:	learn: 0.4795830	total: 125ms	remaining: 9.52s
13:	learn: 0.4709540	total: 134ms	remaining: 9.45s
14:	learn: 0.4635526	total: 143ms	remaining: 9.39s
15:	learn: 0.4563060	total: 151ms	remaining: 9.27s
16:	learn: 0.4496225	total: 159ms	remaining: 9.21s
17:	learn: 0.4438195	total: 167ms	remaining: 9.1s
18:	learn: 0.4380916	total: 176ms	remaining: 9.1s
19:	learn: 0

(0.5033851260480157, 0.5042378458809054, 0.5027155406758806)

#### Let's try second approach with oversampling.

In [46]:
df_tr = pd.read_csv("H:/Learnings/Data Science/Hackathon/HR analytics/train_jqd04QH.csv")
df_te =  pd.read_csv("H:/Learnings/Data Science/Hackathon/HR analytics/test_KaymcHn.csv")

In [47]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             18359 non-null  int64  
 1   city                    18359 non-null  object 
 2   city_development_index  18359 non-null  float64
 3   gender                  14261 non-null  object 
 4   relevent_experience     18359 non-null  object 
 5   enrolled_university     18017 non-null  object 
 6   education_level         17902 non-null  object 
 7   major_discipline        15521 non-null  object 
 8   experience              18300 non-null  object 
 9   company_size            13580 non-null  object 
 10  company_type            13320 non-null  object 
 11  last_new_job            17992 non-null  object 
 12  training_hours          18359 non-null  int64  
 13  target                  18359 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [48]:
df_tr.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [49]:
#Copying the dataframes
df_train = df_tr.copy()
df_test = df_te.copy()

In [51]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [53]:
df_train.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [54]:
df_test.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    3388
relevent_experience          0
enrolled_university        279
education_level            395
major_discipline          2393
experience                  44
company_size              4051
company_type              4330
last_new_job               304
training_hours               0
dtype: int64

In [55]:
df_train['experience'].replace({'<1':'1','>20':'20'},inplace=True)
df_test['experience'].replace({'<1':'1','>20':'20'},inplace=True)
df_train['company_size'].replace({'<10':'very small','10/49':'very small','50-99':'very small',
                                  '100-500':'small','500-999':'small',
                                  '1000-4999':'medium',
                                  '5000-9999':'large','10000+':'large'},inplace=True)
df_test['company_size'].replace({'<10':'very small','10/49':'very small','50-99':'very small',
                                  '100-500':'small','500-999':'small',
                                  '1000-4999':'medium',
                                  '5000-9999':'large','10000+':'large'},inplace=True)
df_train['last_new_job'].replace({'>4':'5','never': '0'},inplace=True)
df_test['last_new_job'].replace({'>4':'5','never': '0'},inplace=True)

In [56]:
#Imputing the Null values in Train and Test dataset
df_train['gender'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['gender']))
df_test['gender'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['gender']))
df_train['enrolled_university'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['enrolled_university']))
df_test['enrolled_university'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['enrolled_university']))
df_train['education_level'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['education_level']))
df_test['education_level'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['education_level']))
df_train['major_discipline'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['major_discipline']))
df_test['major_discipline'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['major_discipline']))
df_train['experience'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['experience']))
df_test['experience'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['experience']))
df_train['company_size'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['company_size']))
df_test['company_size'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['company_size']))
df_train['last_new_job'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['last_new_job']))
df_test['last_new_job'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['last_new_job']))
df_train['company_type'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['company_type']))
df_test['company_type'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['company_type']))

In [57]:
df_test.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

In [58]:
df_train.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [59]:
#changing the dtypes of the featues
df_train['experience'] = df_train['experience'].astype('int')
df_test['experience'] = df_test['experience'].astype('int')
df_train['last_new_job'] = df_train['last_new_job'].astype('int')
df_test['last_new_job'] = df_test['last_new_job'].astype('int')

In [60]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             18359 non-null  int64  
 1   city                    18359 non-null  object 
 2   city_development_index  18359 non-null  float64
 3   gender                  18359 non-null  object 
 4   relevent_experience     18359 non-null  object 
 5   enrolled_university     18359 non-null  object 
 6   education_level         18359 non-null  object 
 7   major_discipline        18359 non-null  object 
 8   experience              18359 non-null  int32  
 9   company_size            18359 non-null  object 
 10  company_type            18359 non-null  object 
 11  last_new_job            18359 non-null  int32  
 12  training_hours          18359 non-null  int64  
 13  target                  18359 non-null  int64  
dtypes: float64(1), int32(2), int64(3), obj

In [61]:
df_train.drop(['city'],1,inplace=True)
df_test.drop(['city'],1,inplace=True)

In [64]:
df_train = pd.get_dummies(data=df_train,drop_first=True)
df_test = pd.get_dummies(data=df_test,drop_first=True)

In [65]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 28 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   enrollee_id                                 18359 non-null  int64  
 1   city_development_index                      18359 non-null  float64
 2   experience                                  18359 non-null  int32  
 3   last_new_job                                18359 non-null  int32  
 4   training_hours                              18359 non-null  int64  
 5   target                                      18359 non-null  int64  
 6   gender_Male                                 18359 non-null  uint8  
 7   gender_Other                                18359 non-null  uint8  
 8   relevent_experience_No relevent experience  18359 non-null  uint8  
 9   enrolled_university_Part time course        18359 non-null  uint8  
 10  enrolled_u

In [66]:
df_train.drop(['enrollee_id'],1,inplace=True)

In [67]:
#oversampling the data
X=df_train.drop(columns=["target"])
y=df_train["target"]
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=123)

In [75]:
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('ADB', AdaBoostClassifier()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('CAT', CatBoostClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    model.fit(X_train, y_train)
    y_test_pred_new=model.predict_proba(X_test)[:,1]
    ac=metrics.roc_auc_score(y_test,y_test_pred_new)   
    results.append(ac)
    names.append(name)
    msg = "%s: %f " % (name, ac)
    print(msg)

C:\Users\Admin -\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LR: 0.838083 
ADB: 0.873716 
GB: 0.885390 
Learning rate set to 0.041893
0:	learn: 0.6818725	total: 44.6ms	remaining: 44.6s
1:	learn: 0.6715265	total: 53.9ms	remaining: 26.9s
2:	learn: 0.6613832	total: 63.2ms	remaining: 21s
3:	learn: 0.6516534	total: 72.5ms	remaining: 18.1s
4:	learn: 0.6437139	total: 81.6ms	remaining: 16.2s
5:	learn: 0.6363956	total: 91ms	remaining: 15.1s
6:	learn: 0.6285038	total: 99.8ms	remaining: 14.2s
7:	learn: 0.6211665	total: 109ms	remaining: 13.5s
8:	learn: 0.6158123	total: 118ms	remaining: 13s
9:	learn: 0.6097734	total: 127ms	remaining: 12.6s
10:	learn: 0.6037689	total: 137ms	remaining: 12.3s
11:	learn: 0.5993957	total: 146ms	remaining: 12s
12:	learn: 0.5943510	total: 155ms	remaining: 11.8s
13:	learn: 0.5889916	total: 165ms	remaining: 11.6s
14:	learn: 0.5841423	total: 175ms	remaining: 11.5s
15:	learn: 0.5794773	total: 187ms	remaining: 11.5s
16:	learn: 0.5750892	total: 196ms	remaining: 11.3s
17:	learn: 0.5721240	total: 206ms	remaining: 11.2s
18:	learn: 0.5685129

In [76]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15021 entries, 0 to 15020
Data columns (total 27 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   enrollee_id                                 15021 non-null  int64  
 1   city_development_index                      15021 non-null  float64
 2   experience                                  15021 non-null  int32  
 3   last_new_job                                15021 non-null  int32  
 4   training_hours                              15021 non-null  int64  
 5   gender_Male                                 15021 non-null  uint8  
 6   gender_Other                                15021 non-null  uint8  
 7   relevent_experience_No relevent experience  15021 non-null  uint8  
 8   enrolled_university_Part time course        15021 non-null  uint8  
 9   enrolled_university_no_enrollment           15021 non-null  uint8  
 10  education_

In [87]:
cb  = CatBoostClassifier()
cb.fit(X_smote, y_smote)

Learning rate set to 0.044625
0:	learn: 0.6811140	total: 14.9ms	remaining: 14.9s
1:	learn: 0.6700867	total: 26.4ms	remaining: 13.2s
2:	learn: 0.6594418	total: 39.6ms	remaining: 13.2s
3:	learn: 0.6501786	total: 50.7ms	remaining: 12.6s
4:	learn: 0.6412193	total: 61.5ms	remaining: 12.2s
5:	learn: 0.6332298	total: 77.2ms	remaining: 12.8s
6:	learn: 0.6253811	total: 87.5ms	remaining: 12.4s
7:	learn: 0.6182926	total: 99ms	remaining: 12.3s
8:	learn: 0.6131880	total: 111ms	remaining: 12.3s
9:	learn: 0.6075503	total: 123ms	remaining: 12.2s
10:	learn: 0.6024069	total: 135ms	remaining: 12.1s
11:	learn: 0.5969066	total: 147ms	remaining: 12.1s
12:	learn: 0.5910228	total: 156ms	remaining: 11.8s
13:	learn: 0.5852246	total: 169ms	remaining: 11.9s
14:	learn: 0.5809435	total: 183ms	remaining: 12s
15:	learn: 0.5769292	total: 196ms	remaining: 12s
16:	learn: 0.5725541	total: 212ms	remaining: 12.3s
17:	learn: 0.5673605	total: 225ms	remaining: 12.3s
18:	learn: 0.5632327	total: 237ms	remaining: 12.2s
19:	learn

In [94]:
#predicting for test set
x_test = df_test.drop(columns=["enrollee_id"])
y_pred = cb.predict(x_test)

In [95]:
y_pred = y_pred.reshape(15021,1)

In [96]:
df_test['target'] = y_pred

In [98]:
df_test['target'] = df_test['target'].astype(int)

In [99]:
submit = df_test[["enrollee_id","target"]].copy()

In [100]:
submit.to_csv('H:/Learnings/Data Science/Hackathon/HR analytics/sample_submission_2.csv', index=False)